<h3> An Awesome Tricky SQL Logic | Complex SQL 10 </h3>

In [0]:
%sql
create table tasks (
date_value date,
state varchar(10)
);

insert into tasks  values ('2019-01-01','success'),('2019-01-02','success'),('2019-01-03','success'),('2019-01-04','fail')
,('2019-01-05','fail'),('2019-01-06','success')

num_affected_rows,num_inserted_rows
6,6


In [0]:
%sql 
select * from tasks

date_value,state
2019-01-01,success
2019-01-02,success
2019-01-03,success
2019-01-04,fail
2019-01-05,fail
2019-01-06,success


<h3> Solution in sql(hive) </h3>

In [0]:
%sql
with cte as(
select *,row_number()over( order by date_value asc)-row_number()over(partition by state order by date_value asc)rn from tasks)
select min(date_value)as start_date,max(date_value)as end_date,state from cte
group by state,rn


start_date,end_date,state
2019-01-01,2019-01-03,success
2019-01-04,2019-01-05,fail
2019-01-06,2019-01-06,success


<h3> Solution in pyspark </h3>

In [0]:
from pyspark.sql.functions import col,row_number,asc,min,max
from pyspark.sql.window import Window

In [0]:
df_t=spark.sql("select * from tasks")
df_t.show()

+----------+-------+
|date_value|  state|
+----------+-------+
|2019-01-01|success|
|2019-01-02|success|
|2019-01-03|success|
|2019-01-04|   fail|
|2019-01-05|   fail|
|2019-01-06|success|
+----------+-------+



In [0]:
win1=Window.partitionBy("state").orderBy(col("date_value").asc())
win2=Window.orderBy(col("date_value").asc())
df_t=df_t.withColumn("rn",row_number().over(win2)-row_number().over(win1))
df_final=df_t.groupBy("state","rn").agg(min("date_value").alias("start_date"),max("date_value").alias("end_date"))
df_final.select("start_date","end_date","state").orderBy("start_date").show()

+----------+----------+-------+
|start_date|  end_date|  state|
+----------+----------+-------+
|2019-01-01|2019-01-03|success|
|2019-01-04|2019-01-05|   fail|
|2019-01-06|2019-01-06|success|
+----------+----------+-------+

